In [9]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from scipy import stats
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt

cwd = Path().resolve()
import chardet  # ! pip install chardet

with open(os.path.join(Path(cwd).parent, 'data', 'look-up', '0003450398_100_Verwaltungseinheiten_KG_2021.csv'), 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
tmp = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'look-up', '0003450398_100_Verwaltungseinheiten_KG_2021.csv'), sep=';', encoding=result['encoding'])
temp2 = tmp[['KG_NUMMER', 'KG', 'GKZ', 'PB', 'PG', 'BL', 'GB']]
t = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'look-up', 'districts_lookup.csv'), sep=';', encoding='latin1')
t.rename(columns={'Gemeinde kennziffer': 'GKZ', 'cc':'CC'}, inplace=True)

In [10]:

def bundesland(k):
    if k == 'Sa':
        return 'Salzburg'
    elif k == 'St':
        return 'Steiermark'
    elif k == 'W':
        return 'Wien'
    elif k == 'T':
        return 'Tirol'
    elif k == 'V':
        return 'Vorarlberg'
    elif k == 'K':
        return 'Kärnten'
    elif k == 'O':
        return 'Oberösterreich'
    elif k == 'N':
        return 'Niederösterreich'
    elif k == 'B':
        return 'Burgenland'

In [11]:
t[t['CC'] >9000].head(10)

,GKZ,Gemeindename,CC,Status,plz,weitere Postleitzahlen
0,10101,Eisenstadt,10101,SR,7000,NaN
1,10201,Rust,10201,SR,7071,NaN
2,10301,Breitenbrunn am Neusiedler See,10301,M,7091,NaN
3,10302,Donnerskirchen,10302,M,7082,NaN
4,10303,Groáhflein,10303,M,7051,NaN
5,10304,Hornstein,10304,M,7053,2491
6,10305,Klingenbach,10305,NaN,7013,NaN
7,10306,Leithaprodersdorf,10306,NaN,2443,NaN
8,10307,Mrbisch am See,10307,NaN,7072,NaN
9,10308,Mllendorf,10308,NaN,7052,NaN


In [12]:
temp2[temp2['BL']=='Wien'].head()
temp3 = temp2.groupby(['KG'], as_index=False).agg({'KG':['size','last'], 'BL':'first', 'PG': 'first', 'GKZ':'first', 'GB':'first'})
temp3.columns = temp3.columns.droplevel(0)
temp3.columns = ['KG.s', 'KG.f', 'BL', 'PG', 'GKZ', 'GB']
temp3[temp3['BL']=='Wien'].head(30)

temp4 = temp3.groupby(['GB'], as_index=False).agg({'GB':['first', 'size'], 'BL':'first', 'PG': 'first', 'GKZ':'first', 'GB':'first'})
temp4[temp4['BL']=='Wien'].shape[0]

12

In [13]:
temp2[(temp2['GKZ']==90001)].head(10)

,KG_NUMMER,KG,GKZ,PB,PG,BL,GB
0,1002,Alsergrund,90001,Wien(Stadt),Wien,Wien,Josefstadt
1,1004,Innere Stadt,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien
2,1005,Josefstadt,90001,Wien(Stadt),Wien,Wien,Josefstadt
3,1006,Landstraße,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien
4,1008,Margarethen,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien
5,1009,Mariahilf,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien
6,1010,Neubau,90001,Wien(Stadt),Wien,Wien,Josefstadt
7,1011,Wieden,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien
8,1101,Favoriten,90001,Wien(Stadt),Wien,Wien,Favoriten
9,1102,Inzersdorf Stadt,90001,Wien(Stadt),Wien,Wien,Favoriten


In [14]:
tmp = temp2.merge(t[['GKZ', 'plz', 'CC']], how='left', on=['GKZ'])
#tmp = tmp.groupby(['KG'], as_index=False).agg({'GKZ':'first', 'PB':'first', 'plz':'first', 'CC':'first'})
tmp.head()

,KG_NUMMER,KG,GKZ,PB,PG,BL,GB,plz,CC
0,1002,Alsergrund,90001,Wien(Stadt),Wien,Wien,Josefstadt,1010.0,90101.0
1,1002,Alsergrund,90001,Wien(Stadt),Wien,Wien,Josefstadt,1020.0,90201.0
2,1002,Alsergrund,90001,Wien(Stadt),Wien,Wien,Josefstadt,1030.0,90301.0
3,1002,Alsergrund,90001,Wien(Stadt),Wien,Wien,Josefstadt,1040.0,90401.0
4,1002,Alsergrund,90001,Wien(Stadt),Wien,Wien,Josefstadt,1050.0,90501.0


In [15]:
tmp[(tmp['GKZ']==90001) &(tmp['KG']=='Albern')].head(40)

,KG_NUMMER,KG,GKZ,PB,PG,BL,GB,plz,CC
368,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1010.0,90101.0
369,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1020.0,90201.0
370,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1030.0,90301.0
371,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1040.0,90401.0
372,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1050.0,90501.0
373,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1060.0,90601.0
374,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1070.0,90701.0
375,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1080.0,90801.0
376,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1090.0,90901.0
377,1109,Albern,90001,Wien(Stadt),Wien,Wien,Innere Stadt Wien,1100.0,91001.0


In [16]:
plz = [1010.0,1020.0,1030.0,1040.0,1050.0,1060.0,1070.0,1080.0,1090.0,1100.0,1110.0,1120.0,1130.0,1140.0,1150.0,1160.0,1170.0,1180.0,1190.0,1200.0,1210.0,1220.0,1230]
gkz= [90101,90201,90301,90401,90501,90601,90701,90801,90901,91001,91101,91201,91301,91401,91501,91601,91701,91801,91901,92001,92101,92201,1230]
len(gkz)

23

In [17]:
lol = tmp.groupby(['KG'], as_index=False).agg({'KG':['size','last'], 'BL':'first', 'PG': 'first', 'GKZ':'first', 'CC':'first'})
lol.columns = lol.columns.droplevel(0)
lol.columns = ['KG.s', 'KG.f', 'BL', 'PG', 'GKZ', 'CC']
print(lol[lol['KG.s']>1].shape[0])
lol[(lol['KG.s']==23)& (lol['BL']=='Wien')].head(40)

763


,KG.s,KG.f,BL,PG,GKZ,CC
88,23,Alsergrund,Wien,Wien,90001,90101.0
222,23,Aspern,Wien,Wien,90001,90101.0
234,23,Atzgersdorf,Wien,Wien,90001,90101.0
253,23,Auhof,Wien,Wien,90001,90101.0
411,23,Breitenlee,Wien,Wien,90001,90101.0
421,23,Brigittenau,Wien,Wien,90001,90101.0
590,23,Donaufeld,Wien,Wien,90001,90101.0
866,23,Erlaa,Wien,Wien,90001,90101.0
901,23,Eßling,Wien,Wien,90001,90101.0
923,23,Favoriten,Wien,Wien,90001,90101.0


In [18]:
l = tmp[tmp['GKZ']==90001]
l.shape[0]

2047

In [19]:

#tmp[tmp['PG']=='Wien'].head(10)

tmp.to_csv('copyinhere.csv', index=False)

In [20]:
###################-

In [26]:
import pandas as pd
import numpy as np
import chardet  # ! pip install chardet

with open(os.path.join(Path(cwd).parent, 'data', 'look-up', 'PLZ_BESTIMMUNGSORT-20220629.csv'), 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
tmp = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'look-up', 'PLZ_BESTIMMUNGSORT-20220629.csv'), sep=';', encoding=result['encoding'])
tmp.rename(columns={'GEMNR': 'GKZ'}, inplace=True)
print(f"len: {tmp.shape[0]}, unique PLZ: {tmp.PLZ.nunique()}, duplicates: {tmp.duplicated('PLZ', keep=False).sum()}")
tmp.head(5)

len: 18867, unique PLZ: 2228, duplicates: 18432


,PLZ,Bestimmungsort,OKZ,Ortschaft,GKZ,GEMNAM
0,1010,Wien,17223,"Wien,Innere Stadt",90001,Wien
1,1020,Wien,17224,"Wien,Leopoldstadt",90001,Wien
2,1030,Wien,17225,"Wien,Landstraße",90001,Wien
3,1040,Wien,17226,"Wien,Wieden",90001,Wien
4,1050,Wien,17227,"Wien,Margareten",90001,Wien


In [27]:
# tmp[tmp.duplicated('PLZ', keep=False) == True]
df2 = tmp.groupby(['PLZ'], as_index=False).agg({'PLZ':'first', 'Bestimmungsort':'first', 'OKZ':'first', 'Ortschaft':'first', 'GKZ':'first', 'GEMNAM': 'first'})
print(f"len: {df2.shape[0]}, unique PLZ: {df2.PLZ.nunique()}, duplicates: {df2.duplicated('PLZ', keep=False).sum()}")
df2.head(5)

len: 2228, unique PLZ: 2228, duplicates: 0


,PLZ,Bestimmungsort,OKZ,Ortschaft,GKZ,GEMNAM
0,1010,Wien,17223,"Wien,Innere Stadt",90001,Wien
1,1020,Wien,17224,"Wien,Leopoldstadt",90001,Wien
2,1030,Wien,17225,"Wien,Landstraße",90001,Wien
3,1040,Wien,17226,"Wien,Wieden",90001,Wien
4,1050,Wien,17227,"Wien,Margareten",90001,Wien


In [30]:
with open(os.path.join(Path(cwd).parent, 'data', 'look-up', 'PLZ_BESTIMMUNGSORT-20220629.csv'), 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
plz = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'look-up', 'PLZ_Verzeichnis-20220629.csv'), sep=';', encoding=result['encoding'])
#plz.rename(columns={'GEMNR': 'GKZ'}, inplace=True)
print(f"len: {plz.shape[0]}, unique PLZ: {plz.PLZ.nunique()}, duplicates: {plz.duplicated('PLZ', keep=False).sum()}")
plz.head(5)


len: 2521, unique PLZ: 2521, duplicates: 0


,PLZ,Ort,Bundesland,gültig ab,gültig bis,NamePLZTyp,intern_extern,adressierbar,Postfach
0,1000,Wien,W,01.08.2009,NaN,PLZ-Postfach,extern,Nein,Ja
1,1004,Wien,W,01.01.1966,NaN,InteressentenPLZ,extern,Nein,Ja
2,1006,Wien,W,01.02.2015,NaN,PLZ-Postfach,extern,Nein,Ja
3,1010,Wien,W,01.01.1966,NaN,PLZ-Adressierung,extern,Ja,Ja
4,1011,Wien Postfach,W,01.01.1966,NaN,PLZ-Postfach,extern,Nein,Ja


In [32]:
df = df2.merge(plz[['PLZ','Ort', 'Bundesland']], on=['PLZ'], how='left')
print(f"len: {df.shape[0]}, unique PLZ: {df.PLZ.nunique()}, duplicates: {df.duplicated('PLZ', keep=False).sum()}")
df['Bundesland'] = df.apply(lambda x: bundesland(x.Bundesland), axis=1)
df.tail(20)

len: 2228, unique PLZ: 2228, duplicates: 0


,PLZ,Bestimmungsort,OKZ,Ortschaft,GKZ,GEMNAM,Ort,Bundesland
2208,9919,Heinfels,16856,Panzendorf,70735,Heinfels,Heinfels,Tirol
2209,9920,Sillian,16875,Arnbach,70728,Sillian,Sillian,Tirol
2210,9931,Außervillgraten,16780,Außervillgraten,70706,Außervillgraten,Außervillgraten,Tirol
2211,9932,Innervillgraten,16795,Innervillgraten,70710,Innervillgraten,Innervillgraten,Tirol
2212,9941,Kartitsch,16809,Hollbruck,70713,Kartitsch,Kartitsch,Tirol
2213,9942,Obertilliach,16852,Bergen,70721,Obertilliach,Obertilliach,Tirol
2214,9943,Untertilliach,16882,Untertilliach,70733,Untertilliach,Untertilliach,Tirol
2215,9951,Ainet,16763,Ainet,70702,Ainet,Ainet,Tirol
2216,9952,St. Johann im Walde,16867,St. Johann im Walde,70725,St. Johann im Walde,St. Johann im Walde,Tirol
2217,9954,Schlaiten,16874,Schlaiten,70727,Schlaiten,Schlaiten,Tirol
